## 텍스트 정보 처리 프로젝트
###### 참고 사이트
ㆍ https://wikidocs.net/22530  
ㆍ https://mr-doosun.tistory.com/24  
ㆍ https://nicola-ml.tistory.com/63  
ㆍ https://ahnsun98.tistory.com/35  (stopword)  

In [1]:
!pip install torchtext==0.9.0 --user    #실행 후 런타임 재시작 필요
!pip install rhinoMorph
import rhinoMorph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.0 MB 16.3 MB/s 
     |████████████████████████████████| 735.5 MB 17 kB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.0+cu116 requires torch==1.13.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.8.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 25.4 MB/s 


ModuleNotFoundError: ignored

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from torchtext.legacy import data
import random
import numpy as np
import pandas as pd
!pip install konlpy
from konlpy.tag import Kkma
kkma = Kkma()

import re
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

import os
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# stopword 처리
from konlpy.tag import Okt as okt
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.0 MB/s 
     |████████████████████████████████| 465 kB 49.6 MB/s 


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
KERNEL_SIZE = 5

def tokenizer(text):
  token = kkma.morphs(text)
  if len(token) < KERNEL_SIZE:
    for i in range(0, KERNEL_SIZE - len(token)):
      token.append("<PAD>") #커널 사이즈보다 문장의 길이가 작은 경우 에러 방지
  return token

1이 긍정, 0이 부정

In [4]:
f = open('/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/Train.txt', 'r')

data = f.readlines()

print(data[0])

5	배공빠르고 굿



In [5]:
# 데이터 셋
train_set = data[0 : int(len(data) * 0.7)]
test_set = data[int(len(data) * 0.7) :]

print(len(train_set))
print(len(test_set))

140000
60000


In [6]:
print(train_set[0][0])
print(train_set[0][2:])
print(test_set[0][0])
print(test_set[0][2:])

5
배공빠르고 굿

4
빠른 배송만족하고 패드 색감이나 감촉 다 좋아요!!



function

In [7]:
print(len(train_set))
print(len(test_set))

print(train_set[0][0])
print(train_set[0][2:])
print(test_set[0][0])
print(test_set[0][2:])

140000
60000
5
배공빠르고 굿

4
빠른 배송만족하고 패드 색감이나 감촉 다 좋아요!!



In [8]:
# 긍정 : 1, 부정 : 0
train_text = []
train_senti = []
test_text = []
test_senti = []

for i in range(0, len(train_set)):
  temp = train_set[i][2:]
  train_text.append(temp[0:len(temp)-1])

  if((train_set[i][0] == '1') or (train_set[i][0] == '2')):
    train_senti.append(0)
  else:
    train_senti.append(1)


for i in range(0, len(test_set)):
  temp = test_set[i][2:]
  test_text.append(temp[0:len(temp)-1])
  
  if((test_set[i][0] == '1') or (test_set[i][0] == '2')):
    test_senti.append(0)
  else:
    test_senti.append(1)


print(train_text[0:20])
print(train_senti[0:20])
print(test_text[0:20])
print(test_senti[0:20])

print(type(train_text[0]))
print(type(train_senti[0]))

#train_text를 파일로 쓰기
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/train_save_data.txt", "w")

for i in range(0, len(train_text)):
  data = train_text[i] + "\n"
  f.write(data)

f.close()

#train_senti를 파일로 쓰기
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/train_save_senti.txt", "w")

for i in range(0, len(train_senti)):
  data = str(train_senti[i]) + "\n"
  f.write(data)

f.close()

['배공빠르고 굿', '택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고', '아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.', '선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..', '민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ', '비추합니다 계란 뒤집을 때 완전 불편해요 ㅠㅠ 코팅도 묻어나고 보기엔 예쁘고 실용적으로 보였는데 생각보다 진짜 별로입니다.', '주문을 11월6에 시켰는데 11월16일에 배송이 왔네요 ㅎㅎㅎ 여기 회사측과는 전화도 안되고 아무런 연락을 받을수가 없으니 답답하신 분들은 다른곳에서 사시는거 추천드립니다', '넉넉한 길이로 주문했는데도 안 맞네요 별로예요', '보폴이 계속 때처럼 나오다가 지금은 안나네요~', '110인데 전문속옷브랜드 위생팬티105보다 작은듯해요. 불편해요. 밴딩부분이 다 신축성없는 일반실로 되어있어 빅사이즈임에도 빅사이즈같지않아요. 입고벗을때 편하게 밴딩부분이 늘어나고 입었을때도 밴딩이 잡아주어야하는데 말이죠.', '사이즈도 딱이고 귀엽고 넘 좋아요 ㅎㅎ', '베이지 색 구매했는데 약간 살색에 가까워요', '화면빨인가봐요;; 노란컬러가 돋보여요;; 저렴한맛에 그냥 씁니다', '별루 ㅏㅛㅇ치ㅗ티ㅓ치ㅗ탛캏타ㅗ티ㅗ티ㅗ티ㅛ티ㅛ티ㅗㅗㅗ치ㅕ치ㅕ쳐ㅣ처ㅣ치ㅓ처ㅣ펴ㅣ쳐ㅣ치ㅕㅐㅛㅌ쵸ㅔ려ㅔㅎ', '촉감도 좋고 무게감이나 핏도 편합니다', '불멍하기좋고 사이즈도 너무 좋아요', '재구매 친구들이 좋은 향 난다고 해요', '실내에서 신는건지 몰랐어요', '재구매 다 좋은데 하나가 이상하네요', '가게를 운영하는 사장님께서 대신 구매 해달라고 하셔서 구매하게되었는데 가게에 달아놓으니까 이쁘네요']
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 

In [9]:
# 피처 벡터화
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5).fit(train_text)
X_train = vect.transform(train_text)

feature_names = vect.get_feature_names()
print("특성 개수:", len(feature_names))
print("처음 20개 특성:\m", feature_names[:20])

특성 개수: 23526
처음 20개 특성:\m ['000', '000원', '000원에', '010', '07', '0점', '10', '100', '1000원', '1000원짜리', '100g', '100개', '100사이즈', '100일', '100장', '100점', '100프로', '105', '105사이즈', '10mm']


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
y_train = pd.Series(train_senti)
scores = cross_val_score(LogisticRegression(solver="liblinear"), X_train, y_train, cv=5)
print('교차 검증 점수:', scores)
print('교차 검증 점수 평균:', scores.mean())
print('\n')

param_grid = {'C': [0.01, 0.1, 1, 3, 5]}
grid = GridSearchCV(LogisticRegression(solver="liblinear"), param_grid, cv=5)
grid.fit(X_train, y_train)
print("최고 교차 검증 점수:", round(grid.best_score_, 3))
print("최적의 매개변수:", grid.best_params_)

교차 검증 점수: [0.87453571 0.87728571 0.87510714 0.87517857 0.87492857]
교차 검증 점수 평균: 0.8754071428571428


최고 교차 검증 점수: 0.875
최적의 매개변수: {'C': 1}


In [11]:
# test 데이터로 검증
X_test = vect.transform(test_text)
y_test = pd.Series(test_senti)
print("테스트 데이터 점수:", grid.score(X_test, y_test))

테스트 데이터 점수: 0.8764833333333333


In [5]:
# 런타임 재시작 필요
# ver.3 (새로운 데이터를 파일에서 읽어 리스트로 생성하여 전달)

# train_data는 gdrive에서 train_save_data 읽기 & stopword(불용어) 처리
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/train_save_data.txt", "r")
lines = f.readlines()
f.close()

# stopword 읽기 #https://ahnsun98.tistory.com/35
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/korean_stopword.txt", "r")
stopwords = f.readlines()
f.close()


word_tokens = []
stop_tokens = []

for stopword in stopwords:
  stop_tokens.append(stopword[:len(stopword) - 1])
for line in lines:
  if line not in stop_tokens:
    word_tokens.append(line[:len(line) - 1])



# print(lines[0:5])
# print(word_tokens[0:5])   # 위의 train_text와 동일
# print(type(word_tokens[0])) # str
# print(stopwords[0:10])
# print(stop_tokens[0:10])

# train_data의 sentimental 데이터 읽기
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/train_save_senti.txt", "r")
sentis = f.readlines()
f.close()

sentiment = []
for senti in sentis:
  sentiment.append(int(senti[:len(senti) - 1]))

# print(sentiment)    #위의 train_senti와 동일
# print(type(sentiment[0]))   # int

# 새로운 데이터로 검증
import rhinoMorph
rn = rhinoMorph.startRhino()
#print('rn\n',rn)
new_input = ''

# 새로운 데이터를 파일에서 읽기
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/final_test_data.txt", "r")
new_inputs = f.readlines()
f.close()

new_input_list = []
for new_input in new_inputs:
  new_input_list.append(new_input)

#print(new_input_list[0])

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5).fit(word_tokens)


# 위 반복
vect = CountVectorizer(min_df=5).fit(word_tokens)
X_train = vect.transform(word_tokens)
y_train = pd.Series(sentiment)

param_grid = {'C': [0.01, 0.1, 1, 3, 5]}
grid = GridSearchCV(LogisticRegression(solver="liblinear"), param_grid, cv=5)
# grid.fit(word_tokens, sentiment)  
grid.fit(X_train, y_train)  

# 위 반복

senti_result = []

# @@@ 데이터 여러 개
for new_input in new_input_list:
  # 입력 데이터 형태소 분석하기
  inputdata = []
  morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
  morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
  inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
  X_input = vect.transform(inputdata)

  #print(float(grid.predict(X_input)))
  result = grid.predict(X_input) # 0은 부정,1은 긍정
  #print(result)
  #print(type(result))
  if result == 0.0:
      #print("부정적인 글입니다")
      senti_result.append(0)
  else:
      #print("긍정적인 글입니다")
      senti_result.append(1)

# @@@ 데이터 여러 개


#print(senti_result[0 : 10])

f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/ProjectResult.txt", "w")
for i in range(0, len(senti_result)):
  data = str(senti_result[i]) + "\n"
  f.write(data)

f.close()

filepath:  /usr/local/lib/python3.8/dist-packages
classpath:  /usr/local/lib/python3.8/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


In [ ]:
# 런타임 재시작 필요
# ver.1

# train_data는 gdrive에서 train_save_data 읽기 & stopword(불용어) 처리
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/train_save_data.txt", "r")
lines = f.readlines()
f.close()

# stopword 읽기 #https://ahnsun98.tistory.com/35
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/korean_stopword.txt", "r")
stopwords = f.readlines()
f.close()


word_tokens = []
stop_tokens = []

for stopword in stopwords:
  stop_tokens.append(stopword[:len(stopword) - 1])
for line in lines:
  if line not in stop_tokens:
    word_tokens.append(line[:len(line) - 1])



# print(lines[0:5])
print(word_tokens[0:5])   # 위의 train_text와 동일
print(type(word_tokens[0])) # str
# print(stopwords[0:10])
print(stop_tokens[0:10])

# train_data의 sentimental 데이터 읽기
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/train_save_senti.txt", "r")
sentis = f.readlines()
f.close()

sentiment = []
for senti in sentis:
  sentiment.append(int(senti[:len(senti) - 1]))

print(sentiment)    #위의 train_senti와 동일
print(type(sentiment[0]))   # int

# 새로운 데이터로 검증
import rhinoMorph
rn = rhinoMorph.startRhino()
print('rn\n',rn)
new_input = '재구매 편리하게 사용할 수 있어요'

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5).fit(word_tokens)

# 입력 데이터 형태소 분석하기
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
X_input = vect.transform(inputdata)

# 위 반복
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vect = CountVectorizer(min_df=5).fit(word_tokens)
X_train = vect.transform(word_tokens)
y_train = pd.Series(sentiment)

param_grid = {'C': [0.01, 0.1, 1, 3, 5]}
grid = GridSearchCV(LogisticRegression(solver="liblinear"), param_grid, cv=5)
# grid.fit(word_tokens, sentiment)  
grid.fit(X_train, y_train)  

# 위 반복

print(float(grid.predict(X_input)))
result = grid.predict(X_input) # 0은 부정,1은 긍정
print(result)
print(type(result))
if result == 0.0:
    print("부정적인 글입니다")
else:
    print("긍정적인 글입니다")

In [ ]:
# 런타임 재시작 필요
# ver.2 (새로운 데이터를 리스트로 받음)

# train_data는 gdrive에서 train_save_data 읽기 & stopword(불용어) 처리
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/train_save_data.txt", "r")
lines = f.readlines()
f.close()

# stopword 읽기 #https://ahnsun98.tistory.com/35
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/korean_stopword.txt", "r")
stopwords = f.readlines()
f.close()


word_tokens = []
stop_tokens = []

for stopword in stopwords:
  stop_tokens.append(stopword[:len(stopword) - 1])
for line in lines:
  if line not in stop_tokens:
    word_tokens.append(line[:len(line) - 1])



# print(lines[0:5])
# print(word_tokens[0:5])   # 위의 train_text와 동일
# print(type(word_tokens[0])) # str
# print(stopwords[0:10])
# print(stop_tokens[0:10])

# train_data의 sentimental 데이터 읽기
f = open("/content/drive/MyDrive/Colab Notebooks/텍스트정보처리/content/train_save_senti.txt", "r")
sentis = f.readlines()
f.close()

sentiment = []
for senti in sentis:
  sentiment.append(int(senti[:len(senti) - 1]))

# print(sentiment)    #위의 train_senti와 동일
# print(type(sentiment[0]))   # int

# 새로운 데이터로 검증
import rhinoMorph
rn = rhinoMorph.startRhino()
print('rn\n',rn)
new_input = ''
new_input_list = ['배송 한달걸림 나빠요', '너무 좋아요 튼튼하고 바퀴도 있어서 가지고 다니기도 편해요 너무 잘 삿어요', '역시 맛있습니다 주기적으로 계속 사먹어요!']

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5).fit(word_tokens)


# 위 반복
vect = CountVectorizer(min_df=5).fit(word_tokens)
X_train = vect.transform(word_tokens)
y_train = pd.Series(sentiment)

param_grid = {'C': [0.01, 0.1, 1, 3, 5]}
grid = GridSearchCV(LogisticRegression(solver="liblinear"), param_grid, cv=5)
# grid.fit(word_tokens, sentiment)  
grid.fit(X_train, y_train)  

# 위 반복



# @@@ 데이터 여러 개
for new_input in new_input_list:
  # 입력 데이터 형태소 분석하기
  inputdata = []
  morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
  morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
  inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
  X_input = vect.transform(inputdata)

  print(float(grid.predict(X_input)))
  result = grid.predict(X_input) # 0은 부정,1은 긍정
  print(result)
  print(type(result))
  if result == 0.0:
      print("부정적인 글입니다")
  else:
      print("긍정적인 글입니다")

# @@@ 데이터 여러 개


